In [26]:
import warnings

# Ignore warning messages during code execution
# The warnings module is imported
# The .filterwarnings() function is called on the warnings module
# The 'ignore' argument is passed to the .filterwarnings() function
# It configures the warnings module to ignore warning messages

warnings.filterwarnings('ignore')


In [27]:
import geopandas as gpd
#from folium import Markup


In [28]:
import geopandas as gpd

# Read the GeoJSON file and load its contents into a GeoDataFrame
gdf = gpd.read_file("dummy_data.geojson")

# Display the first few rows of the GeoDataFrame
gdf.head()


,_id,unique_id,surveyor_id,coordinates,nearby_station,scrap_location,scrap_category,sub_category,scrap_status,verified_scrap,pending_department,custodian,custodian_contact,approx_weight,approx_rate,remarks,latitude,longitude,geometry
0,64fe1b2434c4957cdd206153,mk01,mk01,"22.194581,69.952183",Lalpur,poled-5/67,DEPOT FERROUS,Cl,UNSTACKED SCRAP A,No,Sr. DEN(PRCL),SSE(P)PBR,9574237045,100 MT,4000000,90 R rails,22.194581,69.952183,POINT (69.95218 22.19458)
1,64fe95c4853cdfbcb2fe60f3,test_data,test_data,"-1.777752,36.924870",test_data,test_data,DEPOT FERROUS,Cl,UNSTACKED SCRAP A,Yes,Sr. DEN(PRCL),SSE(P)PBR,test_data,'test_data','test_data','test_data',-1.777752,36.924870,POINT (36.92487 -1.77775)
2,64fe987ce19722204635c61c,XY124,xx124,"21.531127,70.458050",Lalpur,dhasa,DEPOT FERROUS,MS,UNDER AUCTION PROCESS,Yes,Sr. DEN(PRCL),SSE(P)PBR,9574237045,100 MT,4000000,90 R rails,21.531127,70.458050,POINT (70.45805 21.53113)


In [29]:
print(gdf.columns)


Index(['_id', 'unique_id', 'surveyor_id', 'coordinates', 'nearby_station',
       'scrap_location', 'scrap_category', 'sub_category', 'scrap_status',
       'verified_scrap', 'pending_department', 'custodian',
       'custodian_contact', 'approx_weight', 'approx_rate', 'remarks',
       'latitude', 'longitude', 'geometry'],
      dtype='object')


In [31]:
import folium
import requests

# Define a dictionary to map each type to a color
type_colors = {
    'A': 'blue',
    'B': 'green',
    'C': 'red',
    'default': 'gray'  # Default color for unknown types
}

# Create the navigation pane HTML content
selected_columns = ['_id', 'unique_id', 'surveyor_id', 'coordinates', 'nearby_station',
       'scrap_location', 'scrap_category', 'sub_category', 'scrap_status',
       'verified_scrap', 'pending_department', 'custodian',
       'custodian_contact', 'approx_weight', 'approx_rate', 'remarks',
       'latitude', 'longitude', 'geometry']
selected_gdf = gdf[selected_columns]

# Extract the first latitude and longitude values from the selected_gdf DataFrame
latitude = selected_gdf['latitude'].iloc[0]
longitude = selected_gdf['longitude'].iloc[0]

# Create a map object
m = folium.Map(location=[latitude, longitude], zoom_start=10)


# Define a function to fetch the HTML content for the pop-up pane from a URL
def fetch_popup_html(unique_id):
    try:
        url = f'https://dinudashilua.pythonanywhere.com/popup/{unique_id}/'  # Replace with your actual URL
        #print(url)
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return f'<p>Unable to fetch pop-up content for unique_id={unique_id}</p>'
    except Exception as e:
        return f'<p>Error fetching pop-up content for unique_id={unique_id}: {str(e)}</p>'

def fetch_navigation_pane_html(unique_id):
    try:
        url = f'https://dinudashilua.pythonanywhere.com/navigation-pane/{unique_id}/'  # Replace with your actual URL
        #print(url)
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return f'<p>Unable to fetch navigation pane content for unique_id={unique_id}</p>'
    except Exception as e:
        return f'<p>Error fetching navigation pane content for unique_id={unique_id}: {str(e)}</p>'

# Loop through the entries to add markers
for index, row in selected_gdf.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    type_of_item = row['scrap_category']

    # Get the color for the marker based on the type
    marker_color = type_colors.get(type_of_item, type_colors['default'])

    # Fetch the pop-up HTML content for the popup template
    popup_template_html = fetch_popup_html(row['unique_id'])  # Use your function to fetch popup content

    # Fetch the pop-up HTML content for the navigation pane
    navigation_pane_html = fetch_navigation_pane_html(row['unique_id'])  # Use your function to fetch navigation pane content
  
    # Create a new marker for the current point
    marker = folium.Marker(
        location=[latitude, longitude],
        popup=folium.Popup(popup_template_html, max_width=500),  # Use the iframe HTML string to embed the content
        icon=folium.Icon(color=marker_color)
    )

    marker.add_to(m)
    
    

# Save the map to an HTML file
map_path = '/home/dinudashilua/django-app/gis/templates/map.html'
m.save(map_path)

# Display the map
m

